In [1]:
import pymysql
import pandas as pd
import mysql.connector

In [2]:
dbConnection = mysql.connector.connect(
  host="localhost",
  port="3307",
  user="root",
  passwd="",
  database="consultation"
)

In [3]:
dbConnection1 = mysql.connector.connect(
  host="localhost",
  port="3307",
  user="root",
  passwd="",
  database="client"
)

In [4]:
dbConnection2 = mysql.connector.connect(
  host="localhost",
  port="3307",
  user="root",
  passwd="",
  database="facility"
)

In [5]:
data = pd.read_sql("select art_number,person_id,visit_type,tb_status,state from art a join art_appointment b on a.art_id = b.art_id join art_current_status c on a.art_id = c.art_id join art_visit d on a.art_id=d.art_id", dbConnection)

In [6]:
ArtID = pd.read_sql("select art_id,person_id from art",dbConnection)

In [7]:
dataWithArtID = data.merge(ArtID, on="person_id")

In [8]:
visitDate = pd.read_sql("select art_id,state,date from art_current_status",dbConnection)

In [9]:
DatedData = pd.merge(dataWithArtID,visitDate, how='left',left_on=['art_id','state'],right_on=['art_id','state'])

In [10]:
client = pd.read_sql("select person_id,birthdate,sex from person", dbConnection1);

In [11]:
merged = DatedData.merge(client, on="person_id")

In [12]:
trimmedArtNumbers = data['art_number'].str[:8]

In [13]:
out = trimmedArtNumbers.value_counts().index

In [14]:
s = out[0].replace('-','')

In [15]:
facilityConfig = pd.read_sql("select * from facility a join district b on a.district_id = b.district_id join province c on a.province_id = c.province_id",dbConnection2)

In [16]:
facilityConfig.drop(['district_id','province_id','dhis_code','longitude','latitude'], axis=1,inplace=True)

In [17]:
facilityConf = facilityConfig[facilityConfig['facility_id'].str.contains(s)]

In [18]:
province = facilityConf['facility_id'].str[2:4]

In [19]:
district = facilityConf['facility_id'].str[4:6]

In [20]:
faci = facilityConf['facility_id'].str[6:]

In [21]:
facilityConf['FacilityCode'] = faci

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
facilityConf['DistrictCode'] = district

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
facilityConf['ProvinceCode'] = province

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
facilityConf.columns.values[1] = "FacilityName"

In [25]:
facilityConf.columns.values[2] = "DistrictName"

In [26]:
facilityConf.columns.values[3] = "ProvinceName"

In [27]:
facility = facilityConf

In [28]:
n=merged.shape[0]

In [ ]:
## Facility Name
merged['FacilityName']=facility['FacilityName']
faility_name=merged['FacilityName']
faility_name[1:n] = faility_name[0]
merged['FacilityName']=faility_name
## District Name
merged['DistrictName']=facility['DistrictName']
faility_name=merged['DistrictName']
faility_name[1:n] = faility_name[0]
merged['DistrictName']=faility_name
## Province Name
merged['ProvinceName']=facility['ProvinceName']
faility_name=merged['ProvinceName']
faility_name[1:n] = faility_name[0]
merged['ProvinceName']=faility_name
## Facility Code
merged['FacilityCode']=facility['FacilityCode']
faility_name=merged['FacilityCode']
faility_name[1:n] = faility_name[0]
merged['FacilityCode']=faility_name
## District
merged['DistrictCode']=facility['DistrictCode']
district_name=merged['DistrictCode']
district_name[1:n] = district_name[0]
merged['DistrictCode']=district_name
## Province
merged['ProvinceCode']=facility['ProvinceCode']
province_name=merged['ProvinceCode']
province_name[1:n] = province_name[0]
merged['ProvinceCode']=province_name

In [ ]:
a = facility['FacilityName'].str[:45].value_counts().index
b = a[0]

In [ ]:
c = facility['FacilityCode'].str[:20].value_counts().index
d = c[0]

In [ ]:
e = facility['DistrictCode'].str[:20].value_counts().index
f = e[0]

In [ ]:
g = facility['ProvinceCode'].str[:20].value_counts().index
h = g[0]

In [ ]:
i = facility['DistrictName'].str[:45].value_counts().index
j = i[0]

In [ ]:
k = facility['ProvinceName'].str[:45].value_counts().index
l = k[0]

In [ ]:
merged.drop(['person_id','art_id'], axis=1,inplace=True)

In [ ]:
clearMerged = merged.assign(FacilityName=b,DistrictName=j,ProvinceName=l,FacilityCode=d,DistrictCode=f,ProvinceCode=h)

In [ ]:
clearMerged.columns.values[3] = "art_status"

In [ ]:
clearMerged.columns.values[4] = "visit_date"

In [ ]:
clearMerged.head()

In [ ]:
clearMerged.to_csv('/home/ignatious/Documents/ehr/zdatasets/%s.csv'%b, encoding='utf-8', index=False)

In [ ]:
dbConnection.close()
dbConnection1.close()
dbConnection2.close()

# 